In [9]:
##ADD PREDICTIONS
##Factors for prediction
#1. xGF% Comparision
#2. Days of rest
#3. Home Ice
#4. Cap Percentage
#5. Goalie Rankings
#6. X Factor
import pandas as pd
import numpy as np
import requests
from datetime import date, timedelta
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.expand_frame_repr', False)

today = date.today()
print('Games for: '+str(today))

fTable = pd.read_csv('NHL2020.csv',sep=',')
gameday = fTable['Date']==str(today)
prevgame = fTable['Date']==str(today - timedelta(days = 1))
today = fTable[gameday]
prevday = fTable[prevgame]
display(today)

def checkWinner(score1,score2):
    if score1 > score2:
        return True
    else:
        return False

ask = input('View something or add something or clear something? ')
if ask == 'add':
    for i in today.index:
        teamA = today.at[i,'Visitor']
        teamH = today.at[i,'Home']
        today.at[i,'Prediction'] = input('Winner of '+ teamA+' at '+teamH+': ')
            
    display(today) 
    save = input('Save to File? ')
    if save == 'yes':
        fTable[gameday] = today
        fTable.to_csv('NHL2020.csv',index = False)
        print('File Saved')

#elif ask == 'view week':
    #print(fTable[fTable['Date']==str(date.today()+timedelta(days=6))])
    
#elif ask == 'view month':
    #print()
    
#elif ask == 'view team ;code;':
    #print()

elif ask == 'fix goalies':
    gTable = pd.read_csv('GoalieTier.csv')
    gTable.set_index('Team',inplace=True)
    display(gTable)
    team = input('Team Code ')
    display(gTable.loc[team,:])
    swap = input('SWAP goalies or CHANGE starter tier ')
    if swap == 'swap':
        temp = gTable.at[team,'Starter']
        tempt = gTable.at[team,'S Tier']
        gTable.at[team,'Starter'] = gTable.at[team,'Backup']
        gTable.at[team,'S Tier'] = gTable.at[team,'B Tier']
        gTable.at[team,'Backup'] = temp
        gTable.at[team,'B Tier'] = tempt
        display(gTable.loc[team,:])
    elif swap == 'change':
        print(gTable.at[team,'Starter']+'is currently a Tier '+ gTable.at[team,'S Tier']+'. What would you like to change it to?')
        new = input('New Tier ')
        gTable.at[team,'S Tier'] = new
        print('Tier Updated')
        display(gTable.loc[team,:])
    save = input('Save File? ')
    if save == 'yes':
        gTable.to_csv('GoalieTier.csv')
        print('File saved')
    
    
elif ask == 'games today':
    today.to_csv('GamesToday.csv')
    print('File Saved')
elif ask == 'view season':
    display(fTable)
    
elif ask == 'add odds':
    ind = today.index.values[0]
    oTable = pd.read_csv('DailyOdds.csv',sep=',')
    display(oTable) 
    today = today.sort_values(by='Visitor').reset_index(drop = True)
    today['Away Odds'] = oTable['Away Odds']
    today['Home Odds'] = oTable['Home Odds']
    display(today)
    save = input('Save to File?')
    if save == 'yes':
        today.index = today.index+ind
        fTable[gameday] = today
        fTable = fTable.reset_index(drop = True)
        display(today)
        display(fTable)
        fTable.to_csv('NHL2020.csv',index = False)
        print('File Saved')
   
    
elif ask == 'clear today':
    today['Prediction'] = None
    today['Home Odds'] = None
    today['Away Odds'] = None
    today['OT'] = None
    display(today)
    save = input('Save to File? ')
    if save == 'yes':
        display(today)
        fTable[gameday] = today
        fTable.to_csv('NHL2020.csv',index = False)
        print('File Saved')
    
elif ask == 'add results':
    ind = today.index.values[0]
    url = 'https://www.hockey-reference.com/leagues/NHL_2020_games.html'
    html = requests.get(url).content
    rTable = pd.read_html(html)[0]
    rTable = rTable.loc[:,['G','G.1','Unnamed: 5']]
    rTable = rTable[prevgame]
    prevday['OT'] = rTable['Unnamed: 5']
    prevday['Result'] = np.vectorize(checkWinner)(rTable['G'],rTable['G.1'])
    prevday['Result'] = np.where(prevday['Result']==True,prevday['Visitor'],prevday['Home'])
    prevday = prevday.sort_values(by='Visitor')
    display(prevday)
    save = input('Save to File? ')
    if save == 'yes':
        prevday.index = today.index+ind
        fTable[prevgame] = prevday
        fTable.to_csv('NHL2020.csv',index = False)
        print('File Saved')

print('done')

Games for: 2019-10-03


,Date,Visitor,Away Odds,Home,Home Odds,OT,Prediction,Result
4,2019-10-03,Arizona Coyotes,-105.0,Anaheim Ducks,-115.0,NaN,Anaheim,NaN
5,2019-10-03,Boston Bruins,-110.0,Dallas Stars,-110.0,NaN,Dallas,NaN
6,2019-10-03,Buffalo Sabres,160.0,Pittsburgh Penguins,-180.0,NaN,Pittsburgh,NaN
7,2019-10-03,Calgary Flames,110.0,Colorado Avalanche,-130.0,NaN,Calgary,NaN
8,2019-10-03,Florida Panthers,155.0,Tampa Bay Lightning,-175.0,NaN,Tampa Bay,NaN
9,2019-10-03,Minnesota Wild,145.0,Nashville Predators,-165.0,NaN,Nashville,NaN
10,2019-10-03,Montreal Canadiens,130.0,Carolina Hurricanes,-150.0,NaN,Carolina,NaN
11,2019-10-03,Winnipeg Jets,100.0,New York Rangers,-120.0,NaN,Winnipeg,NaN


View something or add something or clear something? fix goalies


,Starter,S Tier,Backup,B Tier
Team,,,,
ANA,Gibson,1,Miller,3
ARI,Raanta,3,Kempuer,3
BOS,Rask,1,Halak,2
BUF,Hutton,3,Ullmark,4
CAR,Mrazek,3,Reimer,4
CBJ,Korpisalo,4,Merzlikins,5
CGY,Riitch,3,Talbot,4
CHI,Crawford,2,Lehner,2
COL,Grubaur,3,Francouz,5


Team Code ARI


Starter     Raanta
S Tier           3
Backup     Kempuer
B Tier           3
Name: ARI, dtype: object

SWAP goalies or CHANGE starter tier swap


Starter    Kempuer
S Tier           3
Backup      Raanta
B Tier           3
Name: ARI, dtype: object

Save File? yes
File saved
done


In [ ]:
#1. xGF% Comparision last two seasons
import pandas as pd
import numpy as np
import requests
pd.set_option('display.expand_frame_repr',False)
#pd.options.display.float_format = '{:,.2%}'.format
def fullTeamName(name):
    name = name.lower()
    if name == 'Anaheim Ducks'.lower() or name == 'Anaheim'.lower() or name == 'Ducks'.lower():
        return 'ANA'
    elif name == 'Arizona Coyotes'.lower() or name == 'Arizona'.lower() or name == 'Coyotes'.lower():
        return 'ARI'
    elif name == 'Boston Bruins'.lower() or name == 'Boston'.lower() or name == 'Bruins'.lower():
        return 'BOS'
    elif name == 'Buffalo Sabres'.lower() or name == 'Buffalo'.lower() or name == 'Sabres'.lower():
        return 'BUF'
    elif name == 'Carolina Hurricanes'.lower() or name == 'Carolina'.lower() or name == 'Hurricanes'.lower():
        return 'CAR'
    elif name == 'Columbus Blue Jackets'.lower() or name == 'Columbus'.lower() or name == 'Blue Jackets'.lower() or name == 'Jackets'.lower():
        return 'CBJ'
    elif name == 'Calgary Flames'.lower() or name == 'Calgary'.lower() or name == 'Flames'.lower():
        return 'CGY'
    elif name == 'Chicago Blackhawks'.lower() or name == 'Chicago'.lower() or name == 'Blackhawks'.lower():
        return 'CHI'
    elif name == 'Colorado Avalanche'.lower() or name == 'Colorado'.lower() or name == 'Avalanche'.lower():
        return 'COL'
    elif name == 'Dallas Stars'.lower() or name == 'Dallas'.lower() or name == 'Stars'.lower():
        return 'DAL'
    elif name == 'Detroit Red Wings'.lower() or name == 'Detroit'.lower() or name == 'Red Wings'.lower() or name == 'Wings'.lower():
        return 'DET'
    elif name == 'Edmonton Oilers'.lower() or name == 'Edmonton'.lower() or name == 'Edmonton'.lower() or name == 'Wings'.lower():
        return 'EDM'
    elif name == 'Florida Panthers'.lower() or name == 'Florida'.lower() or name == 'Panthers'.lower():
        return 'FLA'
    elif name == 'Los Angeles Kings'.lower() or name == 'Los Angeles'.lower() or name == 'Kings'.lower():
        return 'L.A'
    elif name == 'Minnesota Wild'.lower() or name == 'Minnesota'.lower() or name == 'Wild'.lower():
        return 'MIN'
    elif name == 'Montreal Canadiens'.lower() or name == 'Montreal'.lower() or name == 'Canadiens'.lower():
        return 'MTL'
    elif name == 'New Jersey Devils'.lower() or name == 'New Jersey'.lower() or name == 'Devils'.lower():
        return 'N.J'
    elif name == 'Nashville Predators'.lower() or name == 'Nashville'.lower() or name == 'Predators'.lower():
        return 'NSH'
    elif name == 'New York Islanders'.lower() or name == 'Islanders'.lower():
        return 'NYI'
    elif name == 'New York Rangers'.lower() or name == 'Rangers'.lower():
        return 'NYR'
    elif name == 'Ottawa Senators'.lower() or name == 'Ottawa'.lower() or name == 'Senators'.lower():
        return 'OTT'
    elif name == 'Philadelphia Flyers'.lower() or name == 'Philadelphia'.lower() or name == 'Flyers'.lower():
        return 'PHI'
    elif name == 'Pittsburgh Penguins'.lower() or name == 'Pittsburgh'.lower() or name == 'Penguins'.lower():
        return 'PIT'
    elif name == 'San Jose Sharks'.lower() or name == 'San Jose'.lower() or name == 'Sharks'.lower():
        return 'S.J'
    elif name == 'St. Louis Blues'.lower() or name == 'St. Louis'.lower() or name == 'Blues'.lower():
        return 'STL'
    elif name == 'Tampa Bay Lightning'.lower() or name == 'Tampa Bay'.lower() or name == 'Lightning'.lower():
        return 'T.B'
    elif name == 'Toronto Maple Leafs'.lower() or name == 'Toronto'.lower() or name == 'Leafs'.lower():
        return 'TOR'
    elif name == 'Vancouver Canucks'.lower() or name == 'Vancouver'.lower() or name == 'Canucks'.lower():
        return 'VAN'
    elif name == 'Vegas Golden Knights'.lower() or name == 'Vegas'.lower() or name == 'Golden Knights'.lower() or name == 'Knights'.lower():
        return 'VGK'
    elif name == 'Winnipeg Jets'.lower() or name == 'Winnipeg'.lower() or name == 'Jets'.lower():
        return 'WPG'
    elif name == 'Washington Capitals'.lower() or name == 'Washington'.lower() or name == 'Capitals'.lower():
        return 'WSH'
ask = ''
fTable = pd.read_csv('2018-2019 Table.csv',sep=',')
dTable = pd.read_csv('GamesToday.csv',sep=',')
fTable.set_index('index',inplace=True)
for i in range(len(dTable.index)):
    
    home = dTable.at[i,'Home']#input('Home: ')
    home = fullTeamName(home)
    print(home)
    away = dTable.at[i,'Visitor']#input('Away: ')
    away = fullTeamName(away)
    print(away)

    

    status = fTable.loc[away,home]
    perc = 0

    if status == 'Dominate':
        sign = '+'
        perc = 25
    elif status == 'Favorite':
        sign = '+'
        perc = 10
    elif status == 'Slight Edge':
        sign = '+'
        perc = 5
    elif status == 'Lucky':
        sign = '+'
        perc = 2.5

    elif status == 'Unlucky':
        perc = -2.5
    elif status == 'Slight Disadvantage':
        perc = -5
    elif status == 'Underdog':
        perc = -10
    elif status == 'Struggling':
        perc = -25
    else:
        perc = 0

    

    gTable = pd.read_csv('GoalieTier.csv',index_col = 'Team')

    gDiff = gTable.loc[home,'Tier'] - gTable.loc[away,'Tier']
    
    if gDiff == -4:
        gPerc = 20
    elif gDiff == -3:
        gPerc = 15
    elif gDiff == -2:
        gPerc = 10
    elif gDiff == -1:
        gPerc = 5
    
    elif gDiff == 1:
        gPerc = -5
    elif gDiff == 2:
        gPerc = -10
    elif gDiff == 3:
        gPerc = -15
    elif gDiff == 4:
        gPerc = -20

    else:
        gPerc = 0
   
    print(home +' is ' +status+ ' over '+away + ' '+str(perc)+'%')

    hPerc = 5
    print(home +' is at home vs '+away+ ' 5%')
    print(home+' goalie gives '+str(gPerc)+'%')

    hBtoB = ''#input("Home B2B: ")
    if hBtoB == 'yes':
        bPerc = -5
    else:
        bPerc = 0
    
    #input('Grab cap data? ')
    if ask == 'yes':
        url = 'https://www.capfriendly.com/'
        html = requests.get(url).content
        dfList = pd.read_html(html)
        cTable = dfList[3]
        cTable = cTable.loc[:,['TEAM', 'PROJECTED CAP SPACE']]
        cPerc = 0#input('Type the percent bonus from cap space difference')

    else:
        cPerc = 0
    xPerc = 0#input('Type an x-factor bonus/subtract for the home team')
 
    tPerc = hPerc+bPerc+gPerc+perc+xPerc+cPerc
    print(home+' gets a bonus of '+str(tPerc)+'%')

    wPerc = float(.5 * 1+(tPerc/100))
    lPerc = float(1-wPerc)

    print(home + ' chance: '+ str(round(wPerc*100,5)))
    print(away + ' chance: '+ str(round(lPerc*100,5)))
    if wPerc > lPerc:
        print('Prediction: '+ home)
    else:
        print('Prediction: '+ away)

In [ ]:
#FIXER FIXER MISTER MISTER
import pandas as pd
import numpy as np
import requests
from datetime import date,timedelta

url = 'https://www.hockey-reference.com/leagues/NHL_2020_games.html'
html = requests.get(url).content
fTable = pd.read_html(html)[0]
today = date.today()
fTable = fTable[fTable['Date'] == str(today)]
#fTable = fTable.loc[:,['Date','Visitor','Home']]
del fTable['Notes']
display(fTable)
fTable.to_csv('FixToday.csv', index = False)
print('file saved')


In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.expand_frame_repr', False)


today = date.today() + timedelta(days = 4)
yesterday = today - timedelta(days = 1)


fTable = pd.read_csv('NHL2020.csv',sep=',')
fTable.head()
gameday = fTable['Date']==str(today)
prevDay = fTable['Date'] == str(yesterday)
gameday = fTable[gameday]
prevDay = fTable[prevDay]

todayTeams = gameday[['Visitor','Home']]
prevTeams = prevDay[['Visitor','Home']]

print('Games for: '+str(today))
print(todayTeams)
print('Games for: '+ str(yesterday))
print(prevTeams)








In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.expand_frame_repr', False)

teamCodes = ['ANA','ARI','BOS','BUF','CAR','CBJ','CGY','CHI','COL','DAL','DET','EDM','FLA','L.A','MIN','MTL','N.J','NSH','NYI','NYR','OTT','PHI','PIT','S.J','STL','T.B','TOR','VAN','VGK','WPG','WSH']
team = 'ANA'
cTable = pd.DataFrame(index = teamCodes,columns = teamCodes)
cTable = cTable.reset_index()

def teamToCode(opp):
    if 'Av' in opp:
        return 'COL'
    elif 'Bla' in opp:
        return 'CHI'
    elif 'BlueJ' in opp:
        return 'CBJ'
    elif 'Blues' in opp:
        return 'STL'
    elif 'Brui' in opp:
        return 'BOS'
    elif 'Cana' in opp:
        return 'MTL'
    elif 'Canu' in opp:
        return 'VAN'
    elif 'Cap' in opp:
        return 'WSH'
    elif 'Co' in opp:
        return 'ARI'
    elif 'De' in opp:
        return 'N.J'
    elif 'Du' in opp:
        return 'ANA'
    elif 'Fla' in opp:
        return 'CGY'
    elif 'Fly' in opp:
        return 'PHI'
    elif 'Gol' in opp:
        return 'VGK'
    elif 'Hur' in opp:
        return 'CAR'
    elif 'Isl' in opp:
        return 'NYI'
    elif 'Jets' in opp:
        return 'WPG'
    elif 'Kin' in opp:
        return 'L.A'
    elif 'Lig' in opp:
        return 'T.B'
    elif 'Map' in opp:
        return 'TOR'
    elif 'Oil' in opp:
        return 'EDM'
    elif 'Pan' in opp:
        return 'FLA'
    elif 'Pen' in opp:
        return 'PIT'
    elif 'Pred' in opp:
        return 'NSH'
    elif 'Rang' in opp:
        return 'NYR'
    elif 'Red' in opp:
        return 'DET'
    elif 'Sab' in opp:
        return 'BUF'
    elif 'Sen' in opp:
        return 'OTT'
    elif 'Sha' in opp:
        return 'S.J'
    elif 'Sta' in opp:
        return 'DAL'
    elif 'Wild' in opp:
        return 'MIN'
      
stat = 'xGF'
for i in range(len(teamCodes)):
    team = teamCodes[i]
    file = team+stat+' status.csv'
    print(file)
    fTable = pd.read_csv(file,sep = ',')
    fTable['Opp'] = np.vectorize(teamToCode)(fTable['Opp'])
    fTable = fTable.append({'Opp':team,'STATUS':None},ignore_index=True)
    fTable = fTable.sort_values(by = 'Opp').reset_index()
    cTable[team] = fTable['STATUS']    

print(cTable)
cTable.to_csv('2018-2019 Table.csv')